In [1]:
import pandas as pd
import altair as alt

In [2]:
start_date = '2017-01-01'
end_date = '2023-12-31'
years = 7

In [3]:
def to_chf_kwh(df, column='Price_CHF_MWh'):
    return df[column] / 1000 * 100

def to_date(df):
    return pd.PeriodIndex(df.Year.astype(str) + df.Period, freq="Q").to_timestamp()


Referenzmarktpreis 2015–17, gewichtet nach ganzem Produktionsvolumen. Nicht speziell nach PV-Produktion.

In [4]:
middle = pd.read_csv("daten/ogd42_rmp_quartalspreise_2015-2017.csv").rename({'Jahr': 'Year'}, axis=1)
middle['chf_kwh'] = to_chf_kwh(middle) 
middle['Datum'] = to_date(middle) 
middle.head()

,Year,Period,Days,Price_CHF_MWh,Volume_bg_ee_MWh,chf_kwh,Datum
0,2015,Q1,90,49.47,297096,4.947,2015-01-01
1,2015,Q2,91,31.88,586285,3.188,2015-04-01
2,2015,Q3,92,37.95,510491,3.795,2015-07-01
3,2015,Q4,92,52.85,455552,5.285,2015-10-01
4,2016,Q1,91,39.48,463632,3.948,2016-01-01


Referezmarktpreis 2018–23, gewichtet nach PV-Produktionsvolumen:

In [5]:
new = pd.read_csv("daten/ogd60_rmp_quartalspreise.csv")
new['chf_kwh'] = to_chf_kwh(new, column='Price_pv_CHF_MWh')
new['Datum'] = to_date(new)
new.head()

,Year,Period,Days,Price_npv_CHF_MWh,Price_pv_CHF_MWh,Volume_pv_MWh,chf_kwh,Datum
0,2018,Q1,90,58.10,61.75,129448,6.175,2018-01-01
1,2018,Q2,91,43.11,43.09,440983,4.309,2018-04-01
2,2018,Q3,92,64.80,65.08,426576,6.508,2018-07-01
3,2018,Q4,92,74.21,80.33,127504,8.033,2018-10-01
4,2019,Q1,90,55.43,48.97,192671,4.897,2019-01-01


In [6]:
newest = pd.read_csv("daten/newest.csv")
newest['chf_kwh'] = to_chf_kwh(newest)
newest['Datum'] = to_date(newest)
newest

,Year,Period,Price_CHF_MWh,chf_kwh,Datum
0,2023,Q3,71.66,7.166,2023-07-01
1,2023,Q4,87.04,8.704,2023-10-01
2,2024,Q1,61.97,6.197,2024-01-01


In [7]:
d1 = pd.concat([middle, new, newest])[['Datum', 'Year', 'Period', 'chf_kwh']]
d1

,Datum,Year,Period,chf_kwh
0,2015-01-01,2015,Q1,4.947
1,2015-04-01,2015,Q2,3.188
2,2015-07-01,2015,Q3,3.795
3,2015-10-01,2015,Q4,5.285
4,2016-01-01,2016,Q1,3.948
5,2016-04-01,2016,Q2,2.805
6,2016-07-01,2016,Q3,3.348
7,2016-10-01,2016,Q4,6.294
8,2017-01-01,2017,Q1,5.643
9,2017-04-01,2017,Q2,3.557


In [8]:
# PeriodIndex to string
d2 = d1.copy()
d2.loc[d2.Period == 'Q1', 'weight'] = 0.135 / years
d2.loc[d2.Period == 'Q2', 'weight'] = 0.365 / years
d2.loc[d2.Period == 'Q3', 'weight'] = 0.365 / years
d2.loc[d2.Period == 'Q4', 'weight'] = 0.135 / years
d2

,Datum,Year,Period,chf_kwh,weight
0,2015-01-01,2015,Q1,4.947,0.019286
1,2015-04-01,2015,Q2,3.188,0.052143
2,2015-07-01,2015,Q3,3.795,0.052143
3,2015-10-01,2015,Q4,5.285,0.019286
4,2016-01-01,2016,Q1,3.948,0.019286
5,2016-04-01,2016,Q2,2.805,0.052143
6,2016-07-01,2016,Q3,3.348,0.052143
7,2016-10-01,2016,Q4,6.294,0.019286
8,2017-01-01,2017,Q1,5.643,0.019286
9,2017-04-01,2017,Q2,3.557,0.052143


In [9]:
## Mindestpreis == 0 Rappen
d2['chf_kwh_0'] = d2['chf_kwh']
d2.loc[d2.chf_kwh < 0, 'chf_kwh_0'] = 0

## Mindestpreis == 4.6 Rappen
d2['chf_kwh_4_6'] = d2['chf_kwh']
d2.loc[d2.chf_kwh < 4.6, 'chf_kwh_4_6'] = 4.6

## Mindestpreis == 8 Rappen
d2['chf_kwh_8'] = d2['chf_kwh']
d2.loc[d2.chf_kwh < 8, 'chf_kwh_8'] = 8


d3 = d2[['Datum', 'chf_kwh_0', 'chf_kwh_4_6', 'chf_kwh_8', 'weight']]
d3.to_csv("calc/marktpreis_quarter.csv", index=False)
d3.head()

,Datum,chf_kwh_0,chf_kwh_4_6,chf_kwh_8,weight
0,2015-01-01,4.947,4.947,8.0,0.019286
1,2015-04-01,3.188,4.600,8.0,0.052143
2,2015-07-01,3.795,4.600,8.0,0.052143
3,2015-10-01,5.285,5.285,8.0,0.019286
4,2016-01-01,3.948,4.600,8.0,0.019286


In [10]:
alt.Chart(d3.melt(id_vars=['Datum', 'weight']).rename({'value': 'chf_kwh', 'variable': 'mindestpreis'}, axis=1).query('Datum >= "'+start_date+'"')).mark_line(interpolate='step-after').encode(
    alt.X('Datum').axis(format='%Y-%m'),
    alt.Y('chf_kwh', title='CHF/kWh'),
    alt.Color('mindestpreis', title='Mindestpreis in Rappen'),
).properties(width=800, height=400)

alt.Chart(...)

### 7-Jahres-Mittel nach gewichtetem Mindestpreis

In [11]:
d4 = d3.query('Datum >= "'+start_date+'" & Datum <= "'+end_date+'"')
mittel_0 = (d4.chf_kwh_0 * d4.weight).sum()
mittel_0

9.24491

In [12]:
mittel_4_6 = (d4.chf_kwh_4_6 * d4.weight).sum()
mittel_4_6

9.633233571428573

In [13]:
mittel_8 = (d4.chf_kwh_8 * d4.weight).sum()
mittel_8

11.479717857142855

In [14]:
%store mittel_0
%store mittel_4_6
%store mittel_8

Stored 'mittel_0' (float64)
Stored 'mittel_4_6' (float64)
Stored 'mittel_8' (float64)
